<a href="https://colab.research.google.com/github/Troyanovsky/tiny_chain/blob/main/tiny_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tiny Chain
Because LangChain documentation is not easy to navigate + there is too much abstraction, making customization too difficult, I decided to make my own utility functions/classes for LLM usage.

## Install packages


In [ ]:
!pip install openai
!pip install demjson3
!pip install tiktoken
!pip install chromadb
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.1/965.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 66.6 MB/s 

In [ ]:
import openai
import os

# setup openai apk key
openai.api_key = "sk-"

os.environ["OPENAI_API_KEY"] = "sk-"

## Get Response

In [ ]:
import time

def get_API_Response(prompt,system_prompt="You are a helpful assistant"):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 0,
            messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt}
                ]
            )
        response = str(response['choices'][0]['message']['content']).strip()
        return response
    except openai.error.RateLimitError as e:
        time.sleep(5)
        return get_API_Response(prompt)

## Memory
Memory Class
Attributes:
- Raw Messages: All previous messages in a list, in the format of a tuple (role_string, message_string). Role can be "User:" or "AI:"; message can be a string.
- Summary: Summary of all previous messages in one string

Methods:
- Last k words: returns the last messages wtihin the k number of words limit
- Summarize messages: summarizes all messages and store to Summary
- Add message: add one message string, together with the role.
- Delete last message: delete the last message from the list



In [ ]:
def count_words(text):
    return len(text.split())

class SummaryBufferMemory:
    def __init__(self, word_limit=1000):
        self.raw_messages = []
        self.summary = ""
        self.word_count = 0
        self.word_limit = word_limit

    def last_messages(self):
        message_list = []
        word_count = 0
        
        for message_tuple in reversed(self.raw_messages):
            message_word_count = count_words(message_tuple[1])
            if word_count + message_word_count > self.word_limit:
                return "\n".join([f"{role}: {message}" for role, message in message_list])
            else:
                message_list.insert(0,message_tuple)
                word_count += message_word_count

        return "\n".join([f"{role}: {message}" for role, message in message_list])

    def summarize_messages(self):
        if len(self.raw_messages) >= 1:
            messages = "\n".join([f"{role}: {message}" for role, message in self.raw_messages])
        else:
            messages = ""
        prev_summary = self.summary
        summarize_template = f'Summarize the following conversation between an AI and a human in 150 words. The summary must retain all important information. Conversation history:```{messages}``` Previous summary: ```{prev_summary}```'
        summary = get_API_Response(summarize_template)
        self.summary = summary
        self.word_count = 0
        return summary

    def add_message(self, role, message):
        new_message_word_count = count_words(f"{role}: {message}")
        if new_message_word_count + self.word_count >= self.word_limit:
            self.summarize_messages()
        self.raw_messages.append((role, message))
        self.word_count += new_message_word_count

    def delete_last_message(self):
        if len(self.raw_messages) > 0:
            self.raw_messages.pop()

## File loader

In [ ]:
import PyPDF2

class FileLoader:
    def load(self, path):
        if path.endswith('.pdf'):
            pdf_file = open(path, 'rb')
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ''
            for page in pdf_reader.pages:
                text += page.extract_text()
            return text
        elif path.endswith('.txt'):
            with open(path, 'r') as txt_file:
                return txt_file.read()
        else:
            raise ValueError('Unsupported file format')

## Text Splitter

In [ ]:
def text_splitter(string, n_words=500, overlap=50):
    words = string.split()
    sections = []
    
    if n_words >= len(words):
        return [string]
    
    for i in range(0, len(words) - overlap, n_words - overlap):
        section = words[i:i + n_words]
        sections.append(' '.join(section))
    
    return sections

['I am a', 'a good boy.', 'boy. I love', 'love playing football.', "football. I don't", "don't like study."]


## Vector Database
This vectorIndex class provides an interface to work with a database that uses ChromaDB library for vector indexing and searching. 

The class has the following methods:

- __init__(self, documents): Initializes a ChromaDB client and collection with a list of strings (documents)
- add_documents(self, documents): Adds new list of documents to the collection and generates ids for them.
- query_documents(self, query_string, n_results=3): Queries the collection using the given query_string and retrieves n_results number of most relevant documents with their associated ids and distances.
- delete_documents(self, ids): Deletes documents from the collection based on their ids.
- persist(self): Persists the current state of the ChromaDB client to drive.

In [ ]:
import chromadb

class vectorIndex:
    def __init__(self, documents):
        self.client = chromadb.Client(chromadb.config.Settings(chroma_db_impl="duckdb+parquet",persist_directory="database"))
        self.collection = self.client.get_or_create_collection(name="mydb")
        self.last_id = 0
        self.add_documents(documents)
        
    def add_documents(self, documents):
        if len(documents) > 0:
            ids = [str(self.last_id + i) for i in range(len(documents))]
            self.collection.add(documents=documents, ids=ids)
            self.last_id += len(documents)
        
    def query_documents(self, query_string, n_results=3):
        results = self.collection.query(query_texts=[query_string], n_results=n_results)
        documents = results['documents'][0]
        ids = results['ids'][0]
        distances = results['distances'][0]
        result_dict = {'ids': ids, 'documents': documents, 'distances': distances}
        return result_dict
        
    def delete_documents(self, ids):
        self.collection.delete(ids=ids)

    def persist(self):
        self.client.persist()

In [ ]:
# Example usage
texts = ["Hello how are you", "I'm feeling great", "What's the weather like today.",
         "Here is chromadb documentation","from chromdb.config import Settings",
         "collection = client.get_or_create_collection(name=\"mydb\")"]

index = vectorIndex(texts)

result = index.query_documents("chromadb is a vector database", 2)
print(result)

{'ids': ['3', '4'], 'documents': ['Here is chromadb documentation', 'from chromdb.config import Settings'], 'distances': [0.588258683681488, 1.1970349550247192]}
